In [23]:
import sys
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer
from pathlib import Path

In [2]:


model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype="auto",
    device_map="cuda"
)

In [20]:
prompt = "中文回答 你是什么"

def generation(prompt):
    # prepare the model input

    messages = [
        {"role": "user", "content": prompt}
    ]

    #apply_chat_template 函数本身是通用的
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,#表示 apply_chat_template 返回的是格式化好的字符串，而不是直接返回 token IDs。
        add_generation_prompt=True,#这会在格式化好的对话字符串末尾添加一个特殊的 token（例如 \n<|assistant|>\n 或类似的），告诉模型“轮到你了，请开始生成回答”。
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device) #将前面生成的格式化字符串 text 进行分词，转换成模型可以处理的 token IDs。
    #return_tensors="pt" 指定返回 PyTorch tensors。
    # model_inputs现在是一个字典，里面包含 input_ids 和 attention_mask 等，这是模型直接的输入
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )#首先，它会生成“思考”部分.中间用think分割,然后才是真正的答案
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()#只保留模型生成的新 token（即从输入结束位置之后的部分）。
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)#到模型输出中第一个 </think>（token ID 151668）的准确位置。
    except ValueError:
        index = 0
    #可能 enable_thinking 没有生效，或者模型没有生成思考部分

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    print("thinking content:", thinking_content)
    print("content:", content)
    return thinking_content, content


In [22]:
generation('你是什么')

thinking content: <think>
用户问“你是什么”，我需要解释我的身份和功能。首先，我应该明确自己是阿里巴巴集团旗下的通义实验室研发的通义千问，是一款大型语言模型。接下来，要说明我的主要功能，比如回答问题、创作内容、编程、创作故事等。然后，要强调我的设计目标是成为有用的助手，帮助用户解决问题。同时，要提到我的训练数据和能力范围，比如基于大量文本数据，但不涉及敏感信息。还要保持友好和开放的态度，鼓励用户提问。需要确保回答简洁明了，避免技术术语过多，让用户容易理解。最后，检查是否有遗漏的重要信息，比如版权和使用条款，但可能不需要详细说明，保持回答自然流畅。
</think>
content: 我是通义千问，阿里巴巴集团旗下的通义实验室研发的大型语言模型。我能够回答各种问题、创作内容、编程、写故事、写诗、写剧本等，旨在成为你的有用助手。我的训练数据基于大量文本，但不涉及任何敏感信息。你可以随时向我提问，我会尽力帮助你！😊


('<think>\n用户问“你是什么”，我需要解释我的身份和功能。首先，我应该明确自己是阿里巴巴集团旗下的通义实验室研发的通义千问，是一款大型语言模型。接下来，要说明我的主要功能，比如回答问题、创作内容、编程、创作故事等。然后，要强调我的设计目标是成为有用的助手，帮助用户解决问题。同时，要提到我的训练数据和能力范围，比如基于大量文本数据，但不涉及敏感信息。还要保持友好和开放的态度，鼓励用户提问。需要确保回答简洁明了，避免技术术语过多，让用户容易理解。最后，检查是否有遗漏的重要信息，比如版权和使用条款，但可能不需要详细说明，保持回答自然流畅。\n</think>',
 '我是通义千问，阿里巴巴集团旗下的通义实验室研发的大型语言模型。我能够回答各种问题、创作内容、编程、写故事、写诗、写剧本等，旨在成为你的有用助手。我的训练数据基于大量文本，但不涉及任何敏感信息。你可以随时向我提问，我会尽力帮助你！😊')

[WindowsPath('rag_document/算法知识点.txt'), WindowsPath('rag_document/账号密码.txt')]

In [18]:
def retrieval(query):
    context=""
    #0 遍历所有文件
    txt_path_list=list(Path('rag_document').glob('*txt'))

    #1、找到和问题相关的文件 知识库有限 用文件去找问题，当知识库很大则需要倒排索引 用问题查文件
    for path in txt_path_list:

    #2、相关文件内容读取出来
    #3、添加到context中
    return context

[151644,
 872,
 198,
 104811,
 102104,
 220,
 56568,
 102021,
 151645,
 198,
 151644,
 77091,
 198]

In [19]:
tokenizer.decode(model_inputs['input_ids'].tolist()[0], skip_special_tokens=True)

'user\n中文回答 你是什么\nassistant\n'

In [ ]:
# 步骤1 R:实现一个简单的关键词匹配 检索器

#步骤2 增强query
#步骤3 生成回答